# Microsoft Agent Framework - Learning Notebook

**Purpose:** This notebook is designed to explore all the capabilities of the Microsoft Agent Framework for learning and experimentation.

---

## What is Microsoft Agent Framework?

Microsoft Agent Framework is an **open-source development kit** for building AI agents and multi-agent workflows for .NET and Python. It brings together and extends ideas from **Semantic Kernel** and **AutoGen** projects, combining their strengths while adding new capabilities.

### Key Capabilities

| Category | Description |
|----------|-------------|
| **AI Agents** | Individual agents that use LLMs to process user inputs, call tools and MCP servers to perform actions, and generate responses |
| **Workflows** | Graph-based workflows that connect multiple agents and functions to perform complex, multi-step tasks |

### Building Blocks

The framework provides foundational components:
- **Model Clients** - Chat completions and responses (Azure OpenAI, OpenAI, Azure AI)
- **Agent Session** - State management for conversations
- **Context Providers** - Agent memory capabilities
- **Middleware** - Intercepting agent actions
- **MCP Clients** - Tool integration via Model Context Protocol

### When to Use AI Agents

AI agents excel at:
- 🎧 **Customer Support** - Multi-modal queries with tool lookups
- 📚 **Education & Tutoring** - Personalized learning with knowledge bases
- 💻 **Code Generation** - Implementation, reviews, and debugging
- 🔬 **Research Assistance** - Web search, document summarization

### When NOT to Use AI Agents

> *"If you can write a function to handle the task, do that instead of using an AI agent."*

Avoid agents for:
- Highly structured tasks with predefined rules
- Well-defined sequences of operations
- Tasks requiring more than ~20 tools (use workflows instead)

---

## Prerequisites

Before running this notebook:

1. ✅ **Azure subscription** with access to Azure OpenAI
2. ✅ **Azure OpenAI resource** with a deployed model (e.g., `gpt-4o-mini`)
3. ✅ **Azure CLI** installed and authenticated (`az login`)
4. ✅ **`.env` file** with your configuration (see README.md)

## Install Python Packages

To use Microsoft Agent Framework with Azure OpenAI, install the following Python packages:

```bash
pip install agent-framework --pre python-dotenv nest_asyncio
```

> **Note:** `nest_asyncio` is required for Python 3.10 compatibility to allow nested event loops in Jupyter notebooks.

In [ ]:
%pip install agent-framework --pre python-dotenv nest_asyncio

## Load Environment Variables

The `.env` file contains your Azure OpenAI configuration. The `python-dotenv` library loads these variables into the environment so the SDK can access them automatically.

Required variables:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME` - Your deployed model name
- `AZURE_OPENAI_ENDPOINT` - Your Azure OpenAI endpoint URL
- `AZURE_OPENAI_API_KEY` - Your API key (optional if using Azure CLI auth)
- `API_VERSION` - The API version to use

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("Environment variables loaded successfully")

Environment variables loaded successfully


## Create the Agent

First, create a chat client for communicating with Azure OpenAI using the environment variables configured earlier.

Then, create the agent by providing instructions and a name for the agent.

In [8]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="You are good at telling jokes.",
    name="Joker"
)

## Run the Agent

To run the agent, call the `run` method on the agent instance, providing the user input. The agent will return a response object, and accessing the `.text` property provides the text result from the agent.

In [ ]:
async def run_agent():
    result = await agent.run("Tell me a joke about a pirate.")
    print(result.text)

asyncio.run(run_agent())

Why did the pirate go to school?

To improve his "arrrrrrrticulation!"


```markdown
## Run the Agent with Streaming

To run the agent with streaming, call the `run_stream` method on the agent instance, providing the user input. The agent will stream a list of update objects, and accessing the `.text` property on each update object provides the part of the text result contained in that update.
```

In [ ]:
# Stream the response token by token
async def stream_agent():
    async for update in agent.run_stream("Tell me a joke about a pirate."):
        if update.text:
            print(update.text, end="", flush=True)
    print()  # New line after streaming is complete

asyncio.run(stream_agent())

Why did the pirate go to school? 

Because he wanted to improve his "arrrticulation!"


## Run the Agent with ChatMessage Objects

Instead of passing a simple string, you can provide one or more `ChatMessage` objects to the `run` and `run_stream` methods. This gives you more control over the conversation structure, including multi-turn dialogues, system messages, and role-specific content.

## Multimodal Input with ChatMessage

You can pass `ChatMessage` objects with multiple content types, including text and images, to enable multimodal interactions with the agent. This is useful when you want the agent to analyze or respond to visual content along with textual instructions.

In [23]:
from agent_framework import ChatMessage, Content, Role

message = ChatMessage(
    role=Role.USER,
    contents=[
        Content.from_text("Tell me a joke about this image?"),
        Content.from_uri("https://media.gettyimages.com/id/1195994877/vector/democratic-donkey-and-republican-elephant-in-tv-debate.jpg?s=612x612&w=gi&k=20&c=1K-OwflyABXdG_xIbo_n7Ph3CRzI63vGx5G_sKmQz-Y=", media_type="image/jpg")
    ]
)

result = await agent.run(message)
print(result.text)

Why did the donkey and elephant become debate partners?

Because they wanted to show that they could "trunk" out their differences!
